In [17]:
import pandas as pd
import numpy as np
import re

In [30]:
df = pd.read_excel('merged_latest..xlsx')

In [31]:
del df['entireDiv']

In [32]:
df['summary'] = df['description']
df['description'] = np.nan

In [33]:
# df['sal_id'] = 0
# for cat in df['classification'].unique():
#     ind = df[df['classification'] == cat].index
#     new_ind = df[df['classification'] == cat].reset_index().index + 1
#     df.loc[df['classification'] == cat,'sal_id'] = new_ind

df['sal_id'] = df.index + 1

In [34]:
df.columns

Index(['link', 'classification', 'category', 'title', 'company', 'salary',
       'min_salary', 'average_salary', 'max_salary', 'location', 'description',
       'date_posted', 'Date', 'contract', 'summary', 'sal_id'],
      dtype='object')

In [35]:
df['date_posted'].isna().sum()

2221

In [36]:
len(df['location'].unique())

492

In [37]:
df.loc[df['location'] == 'Australia','location'] = 'Other'

In [39]:
df[~df['location'].str.contains(r'^[a-zA-Z\s]+(?=.*[A-Z]{2,})', regex=True)]['location'].unique()

array(['Other'], dtype=object)

In [43]:
stordf = pd.read_excel('sub_city_latandlon.xlsx')

In [66]:
stordf['city'] = stordf['city'].str.replace(' Local Municipality','').str.replace('City Council','').str.strip()

In [87]:
stor_dict = {}

for state in stordf['state'].unique():
    cities = set(stordf[(stordf['state'] == state) & (~stordf['city'].isna())]['city'].unique().tolist())
    city_dict = {}
    for city in cities:
        domain_search_strings = stordf[(stordf['state'] == state) & (stordf['city'] == city)]['domain_search_string'].unique().tolist()
        domain_search_strings = [' '.join(i.split('-')[:-2]) for i in domain_search_strings]
        city_dict[city] = set(domain_search_strings)
    stor_dict[state] = city_dict

In [99]:
for loca in df['location'].unique():
    if loca != 'Other' and loca != np.nan:
        state = loca.split(' ')[-1]
        subu_or_city = ' '.join(loca.split(' ')[:-1]).lower()
        deep_search = 0
        for city_st in stor_dict[state]:
            if city_st.lower() in subu_or_city.lower():
                df.loc[df['location'] == loca,'location'] = city_st.title() + ' ' + state
                deep_serch = 1
                break
        if deep_search == 0:
            for city_st in stor_dict[state]:
                for sub_st in stor_dict[state][city_st]:
                    if sub_st.lower() in subu_or_city.lower():
                        df.loc[df['location'] == loca,'location'] = city_st.title() + ' ' + state
                        deep_search = 1
                        break
                if deep_search == 1:
                    break

In [103]:
df['location'].unique()

array(['Adelaide SA', 'Sydney NSW', 'Brisbane QLD', 'Perth WA',
       'Cairns QLD', 'Other', 'Melbourne VIC', 'Launceston TAS',
       'Hobart TAS', 'Gold Coast QLD',
       'Tennant Creek, Katherine & Northern Australia NT', 'Canberra ACT',
       'Wollongong NSW', 'Darwin NT', 'Toowoomba QLD', 'Geelong VIC',
       'Rockhampton QLD', 'Barongarook, South West Coast VIC',
       'Sunshine Coast QLD', 'Townsville QLD', 'Newcastle NSW',
       'Bunbury WA', 'Esperance WA', 'Albany WA', 'Port Hedland WA',
       'Loxton, Riverland & Murray Mallee SA', 'Mackay QLD',
       'Sunshine Coast Regional QLD', 'Bundaberg QLD', 'Bendigo VIC',
       'Whyalla SA', 'Wodonga VIC', 'Traralgon VIC',
       'Sale, Bairnsdale & Gippsland VIC', 'Geraldton WA',
       'Gladstone QLD', 'Noosa Heads, Sunshine Coast QLD',
       'Winton, Western QLD', 'Broome WA', 'Alice Springs NT',
       'Devonport TAS', 'Wurruk, Bairnsdale & Gippsland VIC',
       'Mount Gambier SA', 'Olympic Dam, Coober Pedy & Outback S

In [109]:
df = df.reindex(columns = ['sal_id', 'link', 'classification', 'category', 'title', 'company', 'salary',
       'min_salary', 'average_salary', 'max_salary', 'location', 'description', 'summary',
       'date_posted', 'Date', 'contract'])

In [110]:
df.columns = df.columns.str.replace(r'\W+', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_').str.lower()

In [111]:
df.columns

Index(['sal_id', 'link', 'classification', 'category', 'title', 'company',
       'salary', 'min_salary', 'average_salary', 'max_salary', 'location',
       'description', 'summary', 'date_posted', 'date', 'contract'],
      dtype='object')

In [112]:
from sqlalchemy import create_engine
import mysql.connector

# Define your database connection parameters
host = "139.99.171.231"
user = "remote"
password = "Peopleai#12345"
database = "peopleai"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

# Close the connection when done
# Define the SQLAlchemy connection string
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

Connected to MySQL database


In [113]:
cursor = connection.cursor()
cursor.execute("SHOW DATABASES")

databases = cursor.fetchall()
for db in databases:
    print(db[0])

information_schema
mysql
peopleai
performance_schema
sys


In [114]:
chunk_size = 10000  # Adjust the chunk size as needed
num_chunks = len(df) // chunk_size + 1

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Delete the existing table if it exists
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS seek_extract")
connection.commit()

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    chunk_df = df.iloc[start_idx:end_idx]
    chunk_df.to_sql(name='seek_extract', con=engine, if_exists='append', index=False)
    print(f'\rprogress: {i+1}/{num_chunks}',end=' ')
print("Data inserted successfully.")

progress: 3/3 Data inserted successfully.
